# Check proxies

In [134]:
import re, os, requests
import concurrent.futures as cf
from tqdm import tqdm

FREE_PROXY_LIST_URL = "https://raw.githubusercontent.com/TheSpeedX/SOCKS-List/master/socks4.txt"
TARGET_URL = "https://api.ipify.org"  # tiny endpoint, returns your IP
CONNECT_TIMEOUT, READ_TIMEOUT = 3, 5
MAX_WORKERS = 64  # tune based on ulimit / OS


In [135]:
def load_proxies(url=FREE_PROXY_LIST_URL):
    txt = requests.get(url, timeout=(5,10)).text
    return list({p for p in re.findall(r"\b(?:\d{1,3}\.){3}\d{1,3}:\d{2,5}\b", txt)})

def load_proxies_from_file(path: str) -> list[str]:
    with open(path, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()
    # keep only host:port patterns
    return [line.strip() for line in lines if line.strip() and ":" in line]


proxies = load_proxies_from_file("out/working_proxies_all.txt")
print(f"Loaded {len(proxies)} proxies")


Loaded 269 proxies


In [136]:
def is_alive(proxy):
    px = {"http": f"{proxy}", "https": f"{proxy}"}
    try:
        r = requests.get(TARGET_URL, proxies=px, timeout=(CONNECT_TIMEOUT, READ_TIMEOUT), allow_redirects=False)
        return proxy if (r.status_code == 200 and r.text.strip()) else None
    except requests.RequestException:
        return None


In [137]:
alive = []
with cf.ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    for res in tqdm(ex.map(is_alive, proxies, chunksize=50), total=len(proxies)):
        if res: alive.append(res)

print(f"Working proxies: {len(alive)}")


100%|██████████| 269/269 [00:12<00:00, 22.28it/s]

Working proxies: 100


In [138]:
os.makedirs("out", exist_ok=True)
with open("out/working_proxies.txt", "w") as f:
    f.write("\n".join(alive))

print("Saved working proxies to out/working_proxies.txt")


Saved working proxies to out/working_proxies.txt
